In [1]:
import pandas as pd
import numpy as np 
import panel as pn
import hvplot.pandas

In [2]:
df = pd.read_csv('co2-data.csv')
df.head()

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Afghanistan,1850,AFG,3752993.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1851,AFG,3769828.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.165,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,Afghanistan,1852,AFG,3787706.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.164,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,Afghanistan,1853,AFG,3806634.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.164,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,Afghanistan,1854,AFG,3825655.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.163,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


In [3]:
df.columns

Index(['country', 'year', 'iso_code', 'population', 'gdp', 'cement_co2',
       'cement_co2_per_capita', 'co2', 'co2_growth_abs', 'co2_growth_prct',
       'co2_including_luc', 'co2_including_luc_growth_abs',
       'co2_including_luc_growth_prct', 'co2_including_luc_per_capita',
       'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2',
       'coal_co2_per_capita', 'consumption_co2', 'consumption_co2_per_capita',
       'consumption_co2_per_gdp', 'cumulative_cement_co2', 'cumulative_co2',
       'cumulative_co2_including_luc', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2',
       'cumulative_oil_co2', 'cumulative_other_co2', 'energy_per_capita',
       'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas_co2',
       'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita',
       'land_use_change_co2', 'land_use_chang

In [4]:
df = df.fillna(0)
df ['gdp_per_capita'] = np.where(df ['population']!= 0, df['gdp']/ df ['population'], 0)

In [5]:
# making the dataframe interactive
idf =df.interactive ()

In [6]:
#define panel widgets]
# slider widget
year_slider = pn.widgets. IntSlider (name='Year slider', start=1750, end=2020, step=5, value=1850)
year_slider

BokehModel(combine_events=True, render_bundle={'docs_json': {'ef90fa8d-b61d-4712-93da-f835557a97be': {'version…

In [7]:
# creating radio button
yaxis_c02 = pn.widgets.RadioButtonGroup(
    name = "y axis",
    options=['co2', 'co2_per_capita',],
    button_type='success'
)

In [8]:
continents = ['World', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

co2_pipeline= (
    idf[(idf.year <=year_slider) & (idf.country.isin(continents))].groupby(["country",'year'])[yaxis_c02].mean().to_frame().reset_index().sort_values(by='year').reset_index(drop=True)

)

In [9]:
co2_pipeline

BokehModel(combine_events=True, render_bundle={'docs_json': {'798cfde3-9f3e-4508-ad65-7f483e2fa75a': {'version…

In [10]:
# co2 emission by continent plot 
co2_plot = co2_pipeline.hvplot(x = 'year',by='country',y = yaxis_c02,title = "CO2 emissions by Continent")


In [11]:
co2_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'36f1bdda-6080-48c7-b779-4b5a3fa1fe19': {'version…

In [12]:
# co2 emission by continent table
co2_table = co2_pipeline.pipe (pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width')
co2_table

BokehModel(combine_events=True, render_bundle={'docs_json': {'5f5c1e06-0337-4675-b71a-a8bf81c32d8a': {'version…

In [13]:
# co2 vs gdp pipeline
co2_vs_gdp_scatter_pipeline = (
    idf[(idf.year == year_slider) & (~ (idf.country.isin(continents)))]
    .groupby(['country','year','gdp_per_capita'])['co2'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)

In [14]:
co2_vs_gdp_scatter_pipeline

BokehModel(combine_events=True, render_bundle={'docs_json': {'4af3c34d-df5c-4475-a50c-b2cbcd9503b3': {'version…

In [15]:
co2_vs_gdp_scatterplot = co2_vs_gdp_scatter_pipeline.hvplot(x='gdp_per_capita',y='co2',by='country',kind='scatter',alpha=0.7,size=80,legend=False,height=500,width=500)
co2_vs_gdp_scatterplot

BokehModel(combine_events=True, render_bundle={'docs_json': {'3c475249-6ba7-47ca-981c-b19d34b06011': {'version…

In [16]:
yaxis_co2_source = pn.widgets.RadioButtonGroup(
    name = "Y axis",
    options = ['coal_co2','oil_co2','gas_co2','cement_co2'],
    button_type = 'success',
)
continents_excl_world = ['Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']


In [17]:
co2_source_bar_pipeline = (
    idf[
        (idf.year==year_slider) &
        (idf.country.isin(continents_excl_world))
    ].groupby(['year','country'])[yaxis_co2_source].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)

In [18]:
co2_source_bar_plot = co2_source_bar_pipeline.hvplot(kind='bar',x='country',y=yaxis_co2_source,title='CO2 Source by Continent')
co2_source_bar_plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'7c64b41b-3b12-472f-989c-4b90b88ba8cf': {'version…

## Creating the dashboard


In [19]:
template = pn.template.FastListTemplate(
    title="World Co2 emissions Dashboard",
    sidebar=[
        pn.pane.Markdown("# CO2 Emissions and Climate Change"),
        pn.pane.Markdown("#### World carbon dioxide (CO2) emissions are a major contributor to climate change. CO2 is released primarily through the burning of fossil fuels such as coal, oil, and natural gas for energy production, transportation, and industrial processes. The excessive accumulation of CO2 in the atmosphere traps heat, causing the Earth's temperature to rise, leading to global warming."),
        pn.pane.Markdown("### Settings"),
        year_slider
    ],
    main=[pn. Row(pn. Column (yaxis_c02,
    co2_plot.panel(width=700), margin=(0,25)),
    co2_table.panel(width=500)),
    pn.Row(pn. Column(co2_vs_gdp_scatterplot.panel(width=600), margin=(0,25)),
    pn.Column(yaxis_co2_source, co2_source_bar_plot.panel (width=600)))], accent_base_color="#88d8b0", header_background="#88d8b0"
)

# template.show()
template.servable()


FastListTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] TemplateActions()
    [browser_info] BrowserInfo()
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [6279653952] Row
        [0] Column(margin=(0, 25))
            [0] RadioButtonGroup(button_type='success', name='y axis', options=['co2', 'co2_per_capita'], value='co2')
            [1] ParamFunction(function, _pane=HoloViews, defer_load=False, width=700)
        [1] ParamFunction(function, _pane=Tabulator, defer_load=False, width=500)
    [6279601200] Row
        [0] Column(margin=(0, 25))
            [0] ParamFunction(function, _pane=HoloViews, defer_load=False, width=600)
        [1] Column
            [0] RadioButtonGroup(button_type='success', name='Y axis', options=['coal_co2', 'oil_co2', ...], value='coal_co2')
            [1] ParamFunction(function, _pane=HoloViews, defer_load=False, width=600)
    [6279605232] Markdown(str)
    [6279602640] Markdown(str)
    [6279597792] Markdown(str)
    [5484226704] IntSlider(end=2020, name='Year slider', start=1750, step=5, value=1850)